In [9]:
import requests
import datetime
import json
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import datetime as dt,date,timedelta
from smtplib import SMTP
import smtplib
from pretty_html_table import build_table
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

## Enter your Postal Code - 
#### Example: 711101

In [11]:
POST_CODE = int(input("Enter Your Postal Code "))

Enter Your Postal Code 110045


## Enter for the Number of Days You Want To Check The Availability
#### Example: 7

In [12]:
numdays = int(input("Enter for the Duration of Days you want to check "))

Enter for the Duration of Days you want to check 5


## Enter the Date from which you Want to check! 
#### Example - 02-05-2021 (dd-mm-yyyy)

In [13]:
date = str(input("Enter the Date, dd-mm-yyy "))
base = pd.to_datetime(date,format='%d-%m-%Y')
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
date_range = [x.strftime("%d-%m-%Y") for x in date_list]

Enter the Date, dd-mm-yyy 03-05-2021


In [14]:
final=[]
for i in date_range:
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?pincode={}&date={}".format(POST_CODE, i)
    response = requests.get(URL)
    result = response.json()
    if result['sessions']:
        df = pd.DataFrame(result['sessions'])
        df= df[['date','center_id','name','state_name','district_name','fee_type','available_capacity','fee','min_age_limit','vaccine']]
        final.append(df)
    else:
        print("No Slot for the {} Available OR Slots are BOOKED!".format(i))
try:
    final = pd.concat(final)
except:
    print("No Slots Available OR Slots are Booked!")

In [15]:
final= final.reset_index(drop=True)

In [16]:
final

,date,center_id,name,state_name,district_name,fee_type,available_capacity,fee,min_age_limit,vaccine
0,03-05-2021,640192,Dada Dev Hospital 5,Delhi,South West Delhi,Free,146,0,45,COVISHIELD
1,03-05-2021,8112,Dada Dev Hospital DH,Delhi,South West Delhi,Free,150,0,45,COVISHIELD
2,03-05-2021,680835,DGD Ramdutt Enclave,Delhi,South West Delhi,Free,20,0,45,COVAXIN
3,03-05-2021,640197,Dada Dev Hospital 6,Delhi,South West Delhi,Free,183,0,45,COVISHIELD
4,03-05-2021,640179,Dada Dev Hospital 4,Delhi,South West Delhi,Free,150,0,45,COVISHIELD
5,03-05-2021,640177,Dada Dev Hospital 3,Delhi,South West Delhi,Free,74,0,45,COVISHIELD
6,04-05-2021,640179,Dada Dev Hospital 4,Delhi,South West Delhi,Free,190,0,45,COVISHIELD
7,04-05-2021,576474,Dada Dev Hospital 2,Delhi,South West Delhi,Free,51,0,45,COVISHIELD
8,04-05-2021,680835,DGD Ramdutt Enclave,Delhi,South West Delhi,Free,43,0,45,COVAXIN
9,04-05-2021,640192,Dada Dev Hospital 5,Delhi,South West Delhi,Free,192,0,45,COVISHIELD


In [8]:
final_html = final.to_html()

In [311]:
email_sender_account = "botguypython@gmail.com"
email_sender_username = "botguypython@gmail.com"
email_sender_password = "password" # Enter your password here
email_smtp_server = "smtp.gmail.com"
email_smtp_port = 587
email_recepients = ["lokeshrth4617@gmail.com","lokesh.rathi@hiveminds.in"]
def SendEmail(final_html):
    email_subject = f"Covid nearest vaccination center and slots availability"
    email_body = '<html><head>  </head><body>'
    email_body += f'{final_html}</h3>' 
    email_body += f'<h3><u>For the Dates with No Data - Either the Slots are booked or are Unavailable!</u></h3>'
    server = smtplib.SMTP(email_smtp_server,email_smtp_port) 
    print(f"Logging in to {email_sender_account}")
    server.starttls() 
    server.login(email_sender_username, email_sender_password)
    for recipient in email_recepients:
        print(f"Sending email to {recipient}")
        message = MIMEMultipart('alternative') 
        message['From'] = email_sender_account 
        message['To'] = recipient 
        message['Subject'] = email_subject 
        message.attach(MIMEText(email_body, 'html'))
        #message.attach(nike)
        server.sendmail(email_sender_account,recipient,message.as_string())
    server.quit()
SendEmail(final_html)